# 2: Fortgeschrittene Datenverarbeitung

- Bereinigung
- falsche Datentypen
- Formatierung 
- Umformung
- Kombinierung von Datensätzen
- Anreicherung


<div style="text-align: center;">
    <img width="30%" src="https://images.ctfassets.net/cnu0m8re1exe/I3pMkGv5UGGmNN1tB4XG0/73edfcdbe6e1637e5e79e0c3a2d25fbb/panda.jpg?fm=jpg&fl=progressive&w=660&h=433&fit=fill" alt="Panda with bamboo" style="min-width: 300px">
    <div><small><em><a href="https://www.discovermagazine.com/planet-earth/how-pandas-survive-on-a-diet-of-only-bamboo">Source</a></em></small></div>
</div>

## Lernpfad

1. **Daten bereinigen**
2. Mit dem Index arbeiten
3. Daten umformen
4. Aggregieren und Gruppieren
5. Zeitreihen

## Daten bereinigen

- Spalten erzeugen, umbenennen, und entfernen
- Typenkonvertierung
- Sortierung

In [ ]:
import pandas as pd

taxis = pd.read_csv('../data/2019_Yellow_Taxi_Trip_Data.csv')
taxis.head()

*Quelle: [NYC Open Data](https://data.cityofnewyork.us/Transportation/2019-Yellow-Taxi-Trip-Data/2upf-qytp) collected via [SODA](https://dev.socrata.com/foundry/data.cityofnewyork.us/2upf-qytp).*

### Spalten entfernen

Alle ID Spalten und die `store_and_fwd_flag` Spalte werden nicht gebraucht.

In [ ]:
mask = taxis.columns.str.contains('id') | (taxis.columns == 'store_and_fwd_flag')
columns_to_drop = taxis.columns[mask]
columns_to_drop

In [ ]:
taxis = taxis.drop(columns=columns_to_drop)
taxis.head()

*Tip: Alternativ könnten wir die Spalten auswählen die wir behalten wollen: `taxis.loc[:,~mask]`.*

### Spalten umbenennen

Vereinfachen wir die Spaltennamen für Ein- und Ausstieg:

In [ ]:
taxis = taxis.rename(
    columns={
        'tpep_pickup_datetime': 'pickup', 
        'tpep_dropoff_datetime': 'dropoff'
    }
)
taxis.columns

### Typenkonvertierung

Sind die Typen der Spalten wirklich so wie sie sein sollten?

In [ ]:
taxis.dtypes

`pickup` and `dropoff` sollten als `datetime` gespeichert sein:

In [ ]:
taxis['pickup'] = pd.to_datetime(taxis['pickup'])
taxis['dropoff'] = pd.to_datetime(taxis['dropoff'])
taxis.dtypes

*Tip: Für numerische Typumwandlung gibt es auch `pd.to_numeric()`. Wir kommen auch noch zum generischen `.astype()`.*

### Neue Spalten

Berechnungen für jede Zeile:
1. Gesamtzeit
2. Trinkgeldanteil
3. Durchschnittsgeschwindigkeit

In [ ]:
taxis['elapsed_time'] = taxis.dropoff - taxis.pickup
cost_before_tip = taxis.total_amount - taxis.tip_amount
taxis['tip_pct'] = taxis.tip_amount / cost_before_tip
taxis['avg_speed'] = taxis.trip_distance / (taxis.elapsed_time.dt.total_seconds() / 60 / 60)

Die neuen Spalten sind ganz rechts:

In [ ]:
taxis.head(2)

### Sortieren

Mit `.sort_values()` kann man Zeilen nach einer beliebigen Anzahl Spalten sortieren.

In [ ]:
taxis.sort_values(['passenger_count', 'trip_distance'], ascending=[False, True]).head()

Um Zeilen mit größten/kleinsten Werten direkt zu finden, nutze `nlargest()` / `nsmallest()`. Die drei Trips mit der längsten Dauer zeigen hier mögliche Datenintegritätsprobleme.

In [ ]:
taxis.nlargest(3, 'elapsed_time')

### Übung 2.1

##### Lade die Daten aus `Meteorite_Landings.csv` ein, benenne die Spalte `mass (g)` um in `mass`, und entferne alle Latitude und Longitude Spalten. Sortiere den Rest nach `mass` in absteigender Reihenfolge.

##### Exportiere die Daten mit `.to_excel`


.

.

.

.

.

.

.

.

.

.

.

.

.


### Lösung

In [ ]:
import pandas as pd

meteorites = pd.read_csv('../data/Meteorite_Landings.csv')
meteorites = meteorites\
    .rename(columns={'mass (g)': 'mass'})\
    .drop(columns=meteorites.columns[-3:])\
    .sort_values('mass', ascending=False)
meteorites.head()

In [ ]:
meteorites.to_excel("meteorites.xlsx")

## Lernpfad

1. Daten bereinigen
2. **Mit dem Index arbeiten**
3. Daten umformen
4. Aggregieren und Gruppieren
5. Zeitreihen

## Forgeschrittenes Indexing

Bis jetzt war der Index nicht mehr als die Zeilennummer. 

Wenn wir eine Spalte mit dtype `datetime` als Index nehmen, haben wir einen DatetimeIndex.

Der Index muss nicht unbedingt unique sein!

In [ ]:
taxis = taxis.set_index('pickup')
taxis.head(3)

Datensatz nach dem Index sortieren:

In [ ]:
taxis = taxis.sort_index(axis="rows")
taxis

*Tip: `taxis.sort_index(axis="columns")` würde nach den Spaltennamen sortieren.*

Jetzt können wir direkt Slices über den Index auswählen:

In [ ]:
taxis.loc['2019-10-23 07:45':'2019-10-23 09:30']

Oder Werte für einen ganzen Tag, indem wir keine Stunden angeben:

In [ ]:
taxis.loc['2019-10-24']

### Index zurücksetzen

Um `pickup` wieder als eine normale Spalte zu behandeln:

In [ ]:
taxis = taxis.reset_index()
taxis.head()

### Übung 2.2

Mit den Daten aus `Meteorite_Landings.csv`,  extrahiere nur das Jahr aus der Spalte `year`, konvertiere es in einen numerischen Datentyp, und erzeuge eine Spalte die besagt ob der Meteorit vor 1970 beobachtet (nicht gefunden) wurde. Setze die `id` Spalte als Index und wähle alle Zeilen mit IDs zwischen 10,036 and 10,040 mit `loc[]`.

**Hinweis 1**: Nutze `year.str.slice()` um an einen Substring zu kommen.

**Hinweis 2**: Sortiere den Index bevor Du `loc[]` verwendest.

**Bonus**: In der `year` Spalte gibt es einen falschen Eintrag. Welcher könnte der sein?

.

.

.

.

.

.

.

.

.

.

.

.

.

.

### Lösung

In [ ]:
import pandas as pd

meteorites = pd.read_csv('../data/Meteorite_Landings.csv').assign(
    year=lambda x: pd.to_numeric(x.year.str.slice(6, 10)),
    observed_pre_1970=lambda x: (x.fall == 'Fell') & (x.year < 1970)
).set_index('id')
meteorites.sort_index().loc[10_036:10_040]

**Notiz**: Eine generelle Alternative für die Jahre ist `pd.to_datetime()`. Müsste hier so benutzt werden:

```python
pd.to_datetime(
    meteorites.year,
    errors='coerce',  # was nicht konvertiert werden kann wird NaT (null)
    format='%m/%d/%Y %I:%M:%S %p'  # das Format des Zeitstrings im Datensatz vor der Umwandlung
)
```

 Mehr Info [hier](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html).

##### **Bonus**

In [ ]:
meteorites.year.describe()

Der Maximumwert liegt in der Zukunft!

In [ ]:
meteorites.query(f'year > {pd.Timestamp("today").year}')

###### Ups! Tatsächlich wurde dieser Meteorit 2010 gefunden (mehr Info [hier](https://www.lpi.usra.edu/meteor/metbull.php?code=57150)).

## Lernpfad

1. Daten bereinigen
2. Mit dem Index arbeiten
3. **Daten umformen**
4. Aggregieren und Gruppieren
5. Zeitreihen

## Daten umformen

Der Taxidatensatz ist schon in einem guten Format für die Analyse - das ist nicht immer so. 

Beispiel ein Datensatz, der tägliche Fluggastzahlen in mehreren Jahren vergleicht.

In [ ]:
tsa = pd.read_csv('../data/tsa_passenger_throughput.csv', parse_dates=['Date'])
tsa.head()

*Quelle: [TSA.gov](https://www.tsa.gov/coronavirus/passenger-throughput)*

Das Ziel ist, das Jahr als Spalte zu bekommen. Damit wird die weitere Analyse sehr erleichtert.

Zuerst ändern wir die Spaltennamen auf Kleinbuchstaben und behalten nur das Jahr.

In [ ]:
def process_col_name(name):
    return name.lower().split()[0]

tsa = tsa.rename(columns=process_col_name)
tsa.head()

### Melting

Melting hilft dabei, die Daten in das sogenannte lange Format zu bekommen.

In [ ]:
tsa_melted = tsa.melt(
    id_vars='date', # Spalte die eine Zeile eindeutig identifiziert
    var_name='year', # Name für die neue Spalte, die melt erzeugt
    value_name='travelers' # Name für neue Spalte mit den Werten der melted columns
)
tsa_melted.sample(5, random_state=1) # Zeige 5 zufällige Einträge

Das Datum stimmt jetzt nicht mehr - die Jahreszahlen für die `date` Spalte müssen aus der `year` Spalte übernommen werden.

In [ ]:
tsa_melted["date"] = pd.to_datetime(tsa_melted.year + tsa_melted.date.astype(str).str[4:])
tsa_melted.sample(5, random_state=1)

Manche Tage waren nicht für alle Jahre im Datensatz und haben deswegen keine Werte:

In [ ]:
tsa_melted.sort_values('date').tail(3)

Diese lassen wir mit `.dropna()` wegfallen:

In [ ]:
tsa_melted = tsa_melted.dropna()
tsa_melted.sort_values('date').tail(3)

### Pivot Tables

Pivot Tables sind in Excel ein wichtiges Konzept, und können auch in Pandas sehr nützlich sein.

Beispiel: Wir wollen die Reisendenzahlen für bestimmte Kalendertage über alle Jahre.

In diesem Fall die ersten zehn Tage im März:

In [ ]:

tsa_pivoted = tsa_melted\
    .assign(day=tsa_melted.date.dt.day)\
    .query('date.dt.month == 3 and day <= 10')\
    .pivot(index='year', columns='day', values='travelers')
tsa_pivoted

### Transponierung

Mit dem `T` Attribut können dann Zeilen und Spalten transponiert werden.

In [ ]:
tsa_pivoted.T

### Merging

Der Flugverkehr ist oft stark von Feiertagen beeinflusst. Wir könnten dies also zum Datensatz hinzufügen um mehr Kontext für Analysen zu haben.

`holidays.csv` enthält die wichtigsten amerikanischen Feiertage in diesem Zeitraum.

In [ ]:
holidays = pd.read_csv('../data/holidays.csv', parse_dates=['date'])
holidays[holidays.date.dt.year == 2019]

Mit unserem Datensatz können wir es mit einem `merge` auf der `date` Spalte verbinden. Das funktioniert etwa wie ein SQL Join.

In [ ]:
tsa_melted_holidays = tsa_melted\
    .merge(holidays, left_on='date', right_on='date', how='left')\
    .sort_values('date')
tsa_melted_holidays.head()

*Tip: Diese Methode ist sehr flexibel, es lohnt sich ein Blick in die [Dokumentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html). Um nur Zeilen anzuhängen gibt es `pd.concat()`.*

Einfluss auf die Zahlen erwarten wir sowohl vor als auch nach den Feiertagen. 

Also markieren wir jeweils den Tag nach einem Feiertag und zwei Tage davor auch als Feiertag, indem wir dort die NaNs auffüllen.

In [ ]:
tsa_melted_holiday_travel = tsa_melted_holidays.copy() 

tsa_melted_holiday_travel['holiday'] = tsa_melted_holidays['holiday']\
            .ffill(limit=1)\
            .bfill(limit=2)

Thanksgiving 2019 war am 28. November, also sind der 26.-29. markiert. Weil wir nur NaNs auffüllen überschreiben wir Weihnachten nicht mit dem Forward Fill von Heiligabend:

In [ ]:
tsa_melted_holiday_travel.query(
    'year == "2019" and '
    '(holiday == "Thanksgiving" or holiday.str.contains("Christmas"))'
)

## Lernpfad

1. Daten bereinigen
2. Mit dem Index arbeiten
3. Daten umformen
4. **Aggregieren und Gruppieren**
5. Zeitreihen

## Aggregieren und Gruppieren

Aggregierungen sind z.B. Statistiken wie `sum()`, `max()`, `mean`.

### Aggregieren mit Pivot Tables
Um den gesammelten Feiertagsflugverkehr über die Jahre zu evaluieren, bauen wir wieder eine Pivot Table:

In [ ]:
tsa_melted_holiday_travel.pivot_table(
    index='year', columns='holiday', 
    values='travelers', aggfunc='sum'
)

Um die Veränderung von Jahr zu Jahr zu sehen, gibt es `.pct_change()`.

In [ ]:
tsa_melted_holiday_travel.pivot_table(
    index='year', columns='holiday', 
    values='travelers', aggfunc='sum'
).pct_change(fill_method=None)

Eine letzte Pivot Table mit Summen über alle Spalten und Zeilen.

Zuerst ein Trick um die Zahlen besser lesbar zu machen:

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)

Dann gruppieren wir die Weihnachtsfeiertage und Neujahr:

In [ ]:
import numpy as np

tsa_melted_holiday_travel['holiday'] = np.where(
        tsa_melted_holiday_travel.holiday.str.contains('Christmas|New Year', regex=True), 
        tsa_melted_holiday_travel.holiday.str.replace('Day|Eve', '', regex=True).str.strip(), 
        tsa_melted_holiday_travel.holiday
    )

Und bauen schließlich die aufgeräumte Tabelle:

In [ ]:
tsa_melted_holiday_travel.pivot_table(
    index='year', columns='holiday', 
    values='travelers', aggfunc='sum', 
    margins=True, margins_name='Total'
)

Vor den weiteren Aufgaben setzen wir die Anzeigeoptionen wieder zurück.

In [ ]:
pd.reset_option('display.float_format')

*Tip: Überblick der verfügbaren Optionen [hier](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).*

### Übung 2.3

Mit den Daten aus `Meteorite_Landings.csv`, erzeuge eine Pivot Table, die die Anzahl der Meteoriten nach Gefunden und Beobachtet aufschlüsselt.

**Bonus**: Beschränke dich auf 2005-2009 (konvertiere wie oben `year` in eine Zahl).

**Bonus**: Zeige zusätzlich das 95. Perzentil der Masse an.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

### Lösung

In [ ]:
import pandas as pd

meteorites = pd.read_csv('../data/Meteorite_Landings.csv').assign(
    year=lambda x: pd.to_numeric(x.year.str.slice(6, 10))
)
meteorites.query('year.between(2005, 2009)').pivot_table(
    index='year', columns='fall', values='mass (g)', 
    aggfunc=['count', lambda x: x.quantile(0.95)]
).rename(columns={'<lambda>': '95th percentile'})

### Gruppierungen
Aggregierungen wie `mean()` oder `describe()` können nicht nur für den ganzen Datensatz, sondern auch für definierte Gruppen berechnet werden.

Summary statistics für jedes Jahr:

In [ ]:
tsa_melted_holiday_travel.groupby('year').describe(include=np.number)

Es gehen auch mehrere Aggregierungen, abhängig von der Spalte.

In [ ]:
tsa_melted_holiday_travel.assign(
    holiday_travelers=lambda x: np.where(~x.holiday.isna(), x.travelers, np.nan),
    non_holiday_travelers=lambda x: np.where(x.holiday.isna(), x.travelers, np.nan)
).groupby('year').agg({
    'holiday_travelers': ['mean', 'std'], 
    'holiday': ['nunique', 'count']
})

Gruppierungen können sehr komplex werden. Dokumentation [hier](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html).

### Übung 2.4

##### Mit den Daten aus `Meteorite_Landings.csv`, berechne Summary Statistics der Massen, aufgeschlüsselt nach Meteoriten die gefunden bzw beobachtet wurden.

### Lösung

In [ ]:
import pandas as pd

meteorites = pd.read_csv('../data/Meteorite_Landings.csv')
meteorites.groupby('fall')['mass (g)'].describe()

## Lernpfad

1. Daten bereinigen
2. Mit dem Index arbeiten
3. Daten umformen
4. Aggregieren und Gruppieren
5. **Zeitreihen**

## Zeitreihen

Pandas hat viele Möglichkeiten,  um Daten mit zeitlicher Komponente intelligent zu analysieren.

- Auswahl von zeitlichen Intervallen
- Veränderungen über Zeit
- Gleitende Zeitfenster für z.B. Mittelwert
- Änderung der Datenfrequenz

### Auswahl

Der `taxis` Datensatz hat Zeitstempel für die Aufnahme und Ablieferung. Zuerst setzen wir die `dropoff` Spalte als den Index und sortieren danach:

In [ ]:
taxis = taxis.set_index('dropoff').sort_index()

Wie vorhin können wir dann Intervalle über den DatetimeIndex auswählen.

In [ ]:
taxis.loc['2019-10-24 12':'2019-10-24 13']

Wenn wir nur die Stunde angeben, ist implizit ein Intervall gemeint.

In [ ]:
taxis.loc['2019-10-24 12']

Wenn wir an dropoffs von verschiedenen Tagen, immer zur gleichen Zeit interessiert sind, nutzen wir `.between_time()` oder `.at_time()`:

In [ ]:
taxis.between_time('12:00', '13:00')

Zurück zu den Fluggastdaten. Hier wählen wir jetzt das Datum als unseren Index.

In [ ]:
tsa_melted_holiday_travel = tsa_melted_holiday_travel.set_index('date')

### Resampling
Mit Resampling geben wir dem Datensatz eine neue Frequenz. 

Dabei müssen wir wählen wie die Daten in den jeweiligen Intervallen kombiniert werden (wie bei `groupby` ):

In [ ]:
tsa_melted_holiday_travel['2019':'2021-Q1'].select_dtypes(include='number')\
    .resample('QE').agg(['sum', 'mean', 'std'])

*Tip: Die möglichen Codes für resample sind [hier](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).* 

### Gleitende Fenster

Pandas unterstützt verschiedene Arten von Zeitfenstern, u.a. gleitende und expandierende. 

<div style="text-align: center;">
<img width="30%" src="https://i0.wp.com/www.untoldwanderlust.com/wp-content/uploads/2017/07/Panda-rolling-around-in-bamboo-in-Chengdu-China.jpg?w=700&ssl=1" alt="Panda Rolling" style="min-width: 300px">
    <div><small><em><a href="https://www.untoldwanderlust.com/chengdu-panda-base/">Source</a></em></small></div>
</div>

Die Benutzung ist ähnlich zu `groupby()`:

In [ ]:
tsa_2020 = tsa_melted_holiday_travel.loc['2020'].copy()

tsa_2020['7D mean'] = tsa_2020.rolling('7D').travelers.mean()
tsa_2020['YTD mean'] = tsa_2020.expanding().travelers.mean()

tsa_2020.head(10)

Um den Output zu verstehen, hilft eine kleine Visualisierung.

Zuerst Setup, damit matplotlib im Notebook SVG als Output erzeugt:

In [ ]:
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats(
    'svg', # output images using SVG format
)

Pandas Dataframes haben eine `.plot()` Methode, die automatisch matplotlib verwendet.

In [ ]:
tsa_2020.plot(title='2020 TSA Traveler Throughput', ylabel='travelers', alpha=0.8)

### Übung 2.5

##### Mit den Taxidaten aus `2019_Yellow_Taxi_Trip_Data.csv` file, resample die Daten zu einer stündlichen Frequenz, basiert auf `dropoff`. Berechne die gesammelte `trip_distance`, `fare_amount`, `tolls_amount`, und `tip_amount`, dann finde die fünf Stunden mit den meisten Trinkgeldern. 

.

.

.

.

.

.

.

.

.

.

.

.



### Lösung

In [ ]:
import pandas as pd

taxis = pd.read_csv(
    '../data/2019_Yellow_Taxi_Trip_Data.csv',
    parse_dates=True, index_col='tpep_dropoff_datetime'
)
taxis.resample('1h')[[
    'trip_distance', 'fare_amount', 'tolls_amount', 'tip_amount'
]].sum().nlargest(5, 'tip_amount')